# Random Environment with Two Stages Repairmen Model

This example demonstrates the simplest random environment model with two operational stages.

## Model Features:
- **Environment**: 2 stages (UP, DOWN)
- **Queueing Network**: 2 stations per stage (Delay + PS Queue)
- **Job Population**: 1 job
- **Simple Transitions**: Basic Markovian transitions between UP/DOWN states
- **Service Rate Switching**: Rates change based on environment stage

## Environment Stages:
1. **Stage1 (UP)**: Rate[0]=2, Rate[1]=1 (fast delay, slow queue)
2. **Stage2 (DOWN)**: Rate[0]=1, Rate[1]=2 (slow delay, fast queue)

## Transition Matrix:
- From Stage1: transition to Stage2 with rate 1.0
- From Stage2: stay in Stage2 with rate 0.5, transition to Stage1 with rate 0.5

This creates a simple two-state environment that switches service characteristics.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.env.*
import jline.solvers.fluid.*
import jline.solvers.ctmc.*
import jline.util.matrix.*

In [ ]:
// Helper function to generate a queueing network for random environment
fun renvGenqn(rates: DoubleArray, N: Int): Network {
    val qnet = Network("qn1")
    
    val node1 = Delay(qnet, "Queue1")
    val node2 = Queue(qnet, "Queue2", SchedStrategy.PS)
    
    val jobclass = ClosedClass(qnet, "Class1", N, node1, 0)
    
    node1.setService(jobclass, Exp(rates[0]))
    node2.setService(jobclass, Exp(rates[1]))
    
    val P = qnet.initRoutingMatrix()
    P.set(jobclass, jobclass, node1, node2, 1.0)
    P.set(jobclass, jobclass, node2, node1, 1.0)
    qnet.link(P)
    
    return qnet
}

In [ ]:
// Model parameters
val N = 1  // Job population
val M = 2  // Number of stations
val E = 2  // Number of environment stages

// Create environment model
val envModel = Env("MyEnv", E)
val envName = arrayOf("Stage1", "Stage2")
val envType = arrayOf("UP", "DOWN")

// Create rate matrix: rates[stage][station]
val rates = arrayOf(
    doubleArrayOf(2.0, 1.0),  // Stage1 (UP): fast delay, slow queue
    doubleArrayOf(1.0, 2.0)   // Stage2 (DOWN): slow delay, fast queue
)

println("Rate matrix:")
rates.forEachIndexed { stage, stageRates ->
    println("${envName[stage]} (${envType[stage]}): ${stageRates.contentToString()}")
}

In [ ]:
// Create queueing networks for each environment stage
val envSubModels = Array(E) { stage ->
    renvGenqn(rates[stage], N)
}

// Add stages to environment model
for (e in 0 until E) {
    envModel.addStage(e, envName[e], envType[e], envSubModels[e])
}

In [ ]:
// Define environment transition rates
val envRates = arrayOf(
    doubleArrayOf(0.0, 1.0),   // From Stage1: transition to Stage2 with rate 1.0
    doubleArrayOf(0.5, 0.5)    // From Stage2: stay in Stage2 (0.5) or go to Stage1 (0.5)
)

println("\nEnvironment transition rates:")
envRates.forEachIndexed { from, transitions ->
    println("From ${envName[from]}: ${transitions.contentToString()}")
}

// Add transitions with exponential distributions
for (e in 0 until E) {
    for (h in 0 until E) {
        if (envRates[e][h] > GlobalConstants.Zero) {
            envModel.addTransition(e, h, Exp(envRates[e][h]))
        }
    }
}

In [ ]:
// Try to display stage table (may not be implemented)
try {
    println("\n=== Stage Table ===")
    val stageTable = envModel.stageTable
    stageTable.print()
} catch (e: Exception) {
    println("\nStage table not available: ${e.message}")
    println("Note: printStageTable() may not be implemented yet")
}

In [ ]:
// Create environment solver with fluid solvers
println("\n=== Creating Environment Solver ===")
try {
    // Create fluid solvers for each submodel
    val solvers = Array(E) { stage ->
        FLD(envSubModels[stage])
    }
    
    // Create environment solver
    val envSolver = ENV(envModel, solvers.toList())
    
    println("Environment solver created successfully")
    
    // Get results
    try {
        val avgTable = envSolver.avgTable
        println("\n=== Average Performance Metrics ===")
        avgTable.print()
    } catch (avgError: Exception) {
        println("Average table error: ${avgError.message}")
    }
    
} catch (e: Exception) {
    println("Error creating environment solver: ${e.message}")
    println("Note: Environment solver features may require specific configurations")
}

In [ ]:
// Alternative: Try with CTMC solvers (as shown in MATLAB version comments)
println("\n=== Alternative: CTMC Solvers ===")
try {
    val ctmcSolvers = Array(E) { stage ->
        CTMC(envSubModels[stage])
    }
    
    val envCTMC = ENV(envModel, ctmcSolvers.toList())
    
    val avgTableCTMC = envCTMC.avgTable
    println("CTMC Results:")
    avgTableCTMC.print()
    
} catch (e: Exception) {
    println("CTMC solver approach failed: ${e.message}")
    println("Note: This is expected and matches the commented nature in the original MATLAB code")
}